In [ ]:
import os, sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import openpyxl
import xlrd
from sklearn.preprocessing import normalize, StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from nilearn.connectome import ConnectivityMeasure
from pandas import DataFrame
import scipy as sc
from scipy import io
from scipy.stats import pearsonr
from os.path import join, exists, dirname, basename
from glob import glob
from brainspace import gradient
from random import randint
import nibabel as nib
import seaborn as sns

# PET-Neurosynth weighted mean

In [ ]:
neurosynth_term_map_list = np.array([np.load(join(neurosynth_path, "terms_ActivationMap", f"{activation_term[i]}_uniformity-test_z_FDR_0.01.npy")) for i in range(len(activation_term))])
neurosynth_term_map_list.shape

In [ ]:
import time
from IPython.display import clear_output
from sklearn.metrics.pairwise import cosine_similarity

"""Path definition"""
pet_path = "X:/ADNI2GO3/PET_preprocessed/"

"""Load Atlas"""
parcel = 200  

# [NOTE] Load Atlas, check Atlas name
Atlas = nib.load(join(atlas_path, f"tpl-MNI152NLin6Asym_res-02_atlas-Schaefer2018_desc-{parcel}Parcels7Networks_dseg.nii.gz")).get_fdata()  # Schaefer
n_roi = int(np.max(Atlas))

mmp_atlas = nib.load(join(atlas_path, 'tpl-MNI152NLin6Asym_res-02_atlas-HCP_dseg.nii.gz'))  # for finding cerebellum region
mmp_atlas = mmp_atlas .get_fdata()
cerebellum_idx = np.where((mmp_atlas == 8)| (mmp_atlas == 47),)

print("Atlas shape: ", Atlas.shape)

neurosynth_term_map_list_abs = np.abs(neurosynth_term_map_list)  # Neurosynth term map processing

"""Extract Atlas mapping PET (AV45 or FBB)"""
for i in range(len(demo["subject_id"])):
    print(f"{i} / {len(demo['subject_id'])}", '', end='', flush=True)
    
    start_time = time.time()
    
    sub_id = demo["subject_id"].iloc[i]
    pt_id = sub_id.split("_")[0]
    session_id = sub_id.split("_")[1]
    
    av45_check = demo["AV45"].iloc[i]
    fbb_check = demo["FBB"].iloc[i]
    
    # [NOTE] Set save path, check save name
    save_path = join(main_path, f'Area/Data/ADNI/{pt_id}/{session_id}')
    if not exists(save_path):
        os.makedirs(save_path)
    
    print(pt_id, session_id, '', end='', flush=True)
    
    if i % 200==0:
        clear_output(wait=True)  # clear cell output
    
    "Check completed session"
    feature_file = glob(join(save_path, f'SUVR_CL_Neurosynth_wmean_Scha{parcel}_Volume.npy'))
    if len(feature_file) != 0:
        print('Already completed') # already
        continue
    
    """Check PET file"""
    nii_av45_file = glob(join(f"{pet_path}/{pt_id}/{session_id}/acq-av45/*task-rest_acq-av45_pet_MNI.nii.gz"))
    nii_fbb_file = glob(join(f"{pet_path}/{pt_id}/{session_id}/acq-fbb/*task-rest_acq-fbb_pet_MNI.nii.gz"))
    
    SUVR_CL_voxel_list = []
    if len(nii_av45_file) !=0:
        nii_file = nii_av45_file
        mid_name = 'av45'
        
        cl_param1 = 196.9  # For centiloid unit calculation 
        cl_param2 = 196.03
        print("AV45", ' ', end='', flush=True)
        
        dt = nib.load(nii_file[0])
    
        # if i % 200==0:
        #     clear_output(wait=True)  # clear cell output
        pet_voxel = dt.get_fdata()
        # print(pet_voxel.shape)
        reference_SUV = np.mean(pet_voxel[cerebellum_idx])

        SUVR_voxel = pet_voxel/reference_SUV
        SUVR_CL_voxel = (cl_param1*SUVR_voxel) - cl_param2
        SUVR_CL_voxel_list.append(SUVR_CL_voxel)
        
    if len(nii_fbb_file) !=0:
        nii_file = nii_fbb_file
        mid_name = 'fbb'
        
        cl_param1 = 159.08 # For centiloid unit calculation 
        cl_param2 = 151.65
        print("FBB", ' ', end='', flush=True)
        
        dt = nib.load(nii_file[0])
    
        # if i % 200==0:
        #     clear_output(wait=True)  # clear cell output
        pet_voxel = dt.get_fdata()
        # print(pet_voxel.shape)
        reference_SUV = np.mean(pet_voxel[cerebellum_idx])

        SUVR_voxel = pet_voxel/reference_SUV
        SUVR_CL_voxel = (cl_param1*SUVR_voxel) - cl_param2
        SUVR_CL_voxel_list.append(SUVR_CL_voxel)
        
    
    if len(nii_av45_file) == 0 and len(nii_fbb_file) == 0:
        print('Pass') # no nii file exist
        continue
    
    SUVR_CL_voxel = np.array((SUVR_CL_voxel_list)).mean(axis=0)  # SUVR centiloid unit
    SUVR_CL_weighted_voxel = np.array([SUVR_CL_voxel/200 * neurosynth_term_map_list_abs[i] for i in range(len(neurosynth_term_map_list_abs))]) 
    
    
    
    """Caculate each ROI noded feature (PET-Neurosynth correlation)"""
    pet_neurosynth_r_feature = np.zeros((n_roi, len(neurosynth_term_map_list)))  # (ROI, Num of term)
    pet_neurosynth_cossim_feature = np.zeros((n_roi, len(neurosynth_term_map_list)))  # (ROI, Num of term)
    pet_neurosynth_wmean_feature = np.zeros((n_roi, len(neurosynth_term_map_list)))  # (ROI, Num of term)
    for roi in range(1, n_roi+1):
        idx = np.where(Atlas==roi)
        
        pet_neurosynth_wmean_feature[roi-1, :] = SUVR_CL_weighted_voxel[:, idx[0], idx[1], idx[2]].sum(axis=1) / neurosynth_term_map_list_abs[:, idx[0], idx[1], idx[2]].sum(axis=1)  # weighted mean
    pet_neurosynth_wmean_feature = np.nan_to_num(pet_neurosynth_wmean_feature)
    
    """Make pet-neurosynth connectivity matrix"""
    pet_neurosynth_wmean_connectivity = np.corrcoef(pet_neurosynth_wmean_feature)
    
    np.save(join(save_path, f'SUVR_CL_Neurosynth_wmean_Scha{parcel}_Volume.npy'), pet_neurosynth_wmean_feature)  # (ROI, Num of term)
    
    np.save(join(save_path, f'SUVR_CL_Neurosynth_wmean_connmat_Scha{parcel}_Volume.npy'), pet_neurosynth_wmean_connectivity)  # (ROI, ROI)
    
    print(f' {time.time() - start_time} seconds!')
    
    # break
    